# Buffer Overflow

By far, the most well-known security vulnerability is the **buffer overflow**. The name says it all. We stuff data into a buffer until it overflows. Why is that a vulnerability? Let's try it and find out. In this notebook, we will explore buffer overflows in detail.

## Overflowable Buffer

In buffo.c, we have the following program.

In [ ]:
#include <stdio.h>

int foo(){
    char buff[128];
    char letter='!';
    int i;
    printf("Enter your name:\n");
    fgets(buff,256,stdin);
    printf("Hello, %sThat's a great name", buff);
    for(i=0;i<10;i++){
        printf("%c",letter);
    }
    printf("\n");
    return 0;
}

int main(){
        foo();
        return 0;
}

Compile the program ("make") and run it ("./buffo"). Type in your name to see the intended behavior of the (admittedly lame) program. The program asks the user for input and then processes their input. You might see code like this in an introductory programming course.

Let's look at the code in detail. "main" calls a function named "foo". Foo creates a 128-character buffer and reads a string into the buffer with the function "fgets". We saw very similar code in a previous notebook.

This code is vulnerable to a buffer overflow. The second argument to "fgets" is the maximum number of characters the function will read. That *should* be 128, since our buffer can't fit any more characters than that. Instead, it's 256. The programmer made a mistake. They changed the size of the buffer without updating the call to fgets. This kind of error is surprisingly common. Try running the program again with an exceptionally long name and see what happens.

## Python Input

Maybe your exceptionally long name made something exciting happen. Maybe not. We're going to experiment with this vulnerable buffer, and filling buffers with data can get a little tedious, especially when using gdb. To save time, we'll write Python scripts to generate the input for us.

Let's start with an infosec classic: filling the buffer with the letter "A". In the file "buffo.py", write this Python script:

In [ ]:
#!/usr/bin/python

buff="A"*100
print(buff)

Make sure you have execute permissions on the Python script. Run it and redirect the output to a file called "input.txt"

./buffo.py > input.txt

If you open input.txt, you should see 100 letter 'A's in there. Very exciting. Now, we can redirect this file as the input to "buffo".

./buffo < input.txt

The program works normally. Boring! Let's get evil!

## Buffer Overflow

Try increasing the number of 'A's the script prints to 130. You should see something like this output:

Hello,   AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA  
That's a great nameAAAAAAAAAA  



Why did this happen? Let's fire up gdb and examine things more closely.

gdb buffo

Disassemble "foo". As usual, the first flew lines of "foo" set up the stack frame. Immediately after the function prologue, you'll see the stack being manipulated to make room for local variables (the "sub" instruction). Immediately after that, we assign the character '!' to the variable "letter". See?

0x080488a8 <+3>:     sub    esp,0x98  
0x080488ae <+9>:     mov    BYTE PTR \[ebp-0xd\],0x21  

No? You don't see it? I don't blame you. There's a lot going on here. The variable "letter" is at memory address ebp-0xd. Let's set a breakpoint here and run the program with our input.

break \*0x080488ae  
run input.txt

Execute that instruction ("si"). We're storing 0x21 there. What is hexadecimal 0x21 in ASCII? Let's check. We can examine variables in ASCII with this command:

x/c $ebp-0xd

You should see that we stored an exclamation mark in "letter". How does that value get changed by our overflow? Let's set a breakpoint after the call to "fgets" and continue execution. Here's where we want to set the breakpoint:

0x080488d7 <+50>:    call   0x804fff0 <fgets>  
0x080488dc <+55>:    add    esp,0x10  

Here's the command to set the breakpoint:

break \*0x080488dc  
continue

What value does "letter" have now? Let's check.

x/c $ebp-0xd

We overwrote it! If you look closely, you'll see that buff starts at memory address ebp-0x8d. You can view the string with this command:

x/s $ebp-0x8d

There are 128 bytes between ebp-0x8d and ebp-0xd (0x8d is 141 and 0xd is 13. 141-13=128.) If we start writing 'A's at ebp-0x8d and continue 130 times, we'll eventually overwrite the value at ebp-0x8d with an 'A'. That's exactly what happened here.

Instead of '!' the program printed 'A'. Not very evil. Let's be more evil!

## Crashing the Program

Let's crash "buffo". Change your input script so it prints 256 'A's. Try running the program on this new input.

./buffo.py > input.txt  
./buffo < input.txt

If you did everything right, "buffo" should crash with the message "Segmentation fault". How did we crash the program?

Again, let's try running the program in gdb with this input. You should see something like this:

Program received signal SIGSEGV, Segmentation fault.  
0x41414141 in ??

What's 0x41414141 in ASCII? That's a big hint as to what's going on. [Go find out](https://www.rapidtables.com/convert/number/hex-to-ascii.html).

Let's set a breakpoint for the ret instruction that is the very last instruction of foo.

break \*0x0804892c  
run < input.txt

What does the return statement do? It pops the return address off the stack and jumps to the instruction at that address to continue executing. The return address on top of the stack. What is on top of the stack right now? Let's find out.

x/wx $esp

There's that value again. What happened? Before you read the next paragraph, see if you can try to guess.

We overflowed the buffer on the stack with so many 'A's that we broke out of the buffer and onto the call stack. We wrote so many 'A's that we overwrote the return address. There's no x86 instructions at memory address 0x41414141, so the program crashes.

This is important! Very important! We can overwrite the return address! We can **inject** a new return address. We can choose the next line of code that gets executed! We'll explore this further in the next notebook. For now, let's make sure you understand how to overflow buffers.

## Exercises

There's only one exercise for this lesson. It's time to be a legit hacker. Are you up to the challenge?

The file "hackme.c" is a friendly little program that asks you for your favorite color and then prints out a polite message with some pretty formatting. Let's break it!

In [ ]:
What is your favorite color?
BLUE
_______________________________________________________
          YOUR FAVORITE COLOR IS BLUE

_______________________________________________________

"hackme.c" contains a overflowable buffer. Modify the Python script in "hackme.py" to produce input that overflows this buffer. Overwrite the values of the variables "fill" and "n". Instead of 55 underscores (\_), make it output 70 hashtags (#). The result of your hack should look something like this:

In [ ]:
What is your favorite color?
######################################################################
          YOUR FAVORITE COLOR IS !!!!!REDACTED!!!!!
######################################################################

It doesn't matter what gets printed in the !!!!!REDACTED!!!!! part. Just make the strings above and below 70 hashtags. The !!!!!REDACTED!!!!! part is redacted because it would give away the answer.

GOOD LUCK, HACKER!